# Předzpracování datasetu SST2

Tento notebook slouží k předzpracování datasetu SST2. Pro dataset jsou vytvořeny augmentované záznamy a předpočítány logity.
Nejprve jsou načteny všechny potřebné knihovny včetně vlastní sbírky objektů a funkcí.

In [37]:
from datasets import load_from_disk, Dataset, concatenate_datasets, ClassLabel, Features, Value, Sequence
from transformers import BertTokenizer, BertForSequenceClassification, BasicTokenizer
from torch.utils.data import  DataLoader
from tqdm.notebook import tqdm
import torch
import copy
import base
import os


Ověření, že GPU je k dispozici a balíček torch je správně nakonfigurován.

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


Konfigurace augmentačních parametrů.
Dataset se bude procházet pětkrát, každý token v záznamu bude na deset procent zamaskován a na třicet procent nahrazen jiným tokenem se stejným POS tagem. Po průchodu všech tokenů v záznamu může dojít s pravděpodobností dvacet procent ke zkrácení záznamu. 

In [ ]:
augmentation_params = {"n_iter": 5, "p_mask":0.1, "p_pos": 0.3, "p_ng":0.2}

Získání datasetu, základního tokenizeru a jeho použití nad datasetem. 

In [ ]:
tokenizer = BasicTokenizer(do_lower_case=True)
DATASET = "sst2"

In [5]:
train_data = load_from_disk(f"~/data/{DATASET}/train")
sentences = list(map(lambda e: e["sentence"], train_data))

Získání POS tagů jednotlivých tokenů v záznamech pro potřeby augmentace.

In [6]:
pos_tag_word_map_list = base.get_pos_tag_word_map(sentences, tokenizer=tokenizer)

In [ ]:
base.reset_seed()

Spuštění procesu augmentace s popsanými parametry.

In [8]:
augmented_datasets = base.get_augmented_dataset(augmentation_params, train_data, pos_tag_word_map_list, tokenizer=tokenizer)

Převedení nových záznamů do dataset objektu.

In [9]:
aug_datasets_formated = []
ds_schema = Features({
    "idx": Value("int32"),
    "sentence": Value("string"),
    "label": ClassLabel(names=["negative", "positive"])
})

for iter in augmented_datasets:
    dataset = Dataset.from_dict(iter)
    dataset = dataset.cast(ds_schema)
    aug_datasets_formated.append(dataset)

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Ověření efektu augmentace, zde např. zamaskované slovo.

In [10]:
print(aug_datasets_formated[4][70])
print(train_data[70])

{'idx': 6575, 'sentence': "in reality it ' s [MASK] tough rock .", 'label': 1}
{'idx': 6575, 'sentence': "in reality it 's one tough rock . ", 'label': 1}


Načtení učitelského modelu a jeho tokenizeru.

In [11]:
tokenizer = BertTokenizer.from_pretrained("gchhablani/bert-base-cased-finetuned-sst2")
model = BertForSequenceClassification.from_pretrained("gchhablani/bert-base-cased-finetuned-sst2", num_labels=2)
model.to(device)
model.eval()

torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/teacher.pth")

config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Předzpracování trénovací části datasetu a výpočet logitů učitelským modelem. Vypočtené logity se přidávají jako nový sloupec datasetu, naopak se odstraňují záznamy ponechané po tokenizeru učitele, které dále nejsou třeba. 

In [12]:
train_dataset = base.prepare_dataset(train_data, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=False)
train_logits = base.generate_logits(train_dataloader, model)
train_dataset = train_dataset.add_column("logits", train_logits)
train_dataset = train_dataset.remove_columns(["token_type_ids", "attention_mask"])
train_dataset.set_format(type="torch", columns=["logits", "labels"], device="cpu")

Tokenizing the provided dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Generating logits for given dataset:   0%|          | 0/421 [00:00<?, ?it/s]

Výpočet správnosti učitelských predikcí nad trénovací částí datasetu.

In [13]:
print(base.check_acc(train_dataset, "Accuracy for base dataset: "))

Calculating accuracy based on the saved logits:   0%|          | 0/53879 [00:00<?, ?it/s]

Accuracy for base dataset:  0.9888268156424581


Stejné kroky pro výpočet logitů jsou provedeny nad každou z pěti augmentovaných variant datasetu. Postupně je skrze záznamy iterováno. Dataset je nejprve tokenizován a následně je předán učitely k inferenci pro získání logitů, které jsou k datasetu uloženy. Odstraněny jsou poté nepotřebné pozůstatky tokenizace. V rámci průběhu zpracování rovnou dochází k filtraci augmentovaných záznamů dle popsaného mechanismu (notebook precompute_logits_10.ipynb).

Takto zpracované datasety se postupně spojují do jednoho celku. 

In [14]:
aug_clean_datasets = []
for dataset in tqdm(aug_datasets_formated, total=(len(aug_datasets_formated)), desc="Processing augmented datasets: "):
    aug_train_dataset = base.prepare_dataset(dataset, tokenizer)
    aug_train_dataloader = DataLoader(aug_train_dataset, batch_size=128, shuffle=False)
    aug_train_logits = base.generate_logits(aug_train_dataloader, model)
    aug_train_dataset = aug_train_dataset.add_column("logits", aug_train_logits)
    aug_train_dataset = aug_train_dataset.remove_columns(["token_type_ids", "attention_mask"])
    aug_train_dataset.set_format(type="torch", columns=["logits", "labels"], device="cpu")

    aug_train_dataset = base.remove_diff_pred_class(train_dataset, aug_train_dataset, pytorch_dataset=False)
    
    aug_train_dataset.reset_format()
    aug_clean_datasets.extend(aug_train_dataset)

Processing augmented datasets:   0%|          | 0/5 [00:00<?, ?it/s]

Tokenizing the provided dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Generating logits for given dataset:   0%|          | 0/421 [00:00<?, ?it/s]

Removing entries from augmented dataset that are different from the base one - based on saved logits:   0%|   …

Tokenizing the provided dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

...

Generating logits for given dataset:   0%|          | 0/421 [00:00<?, ?it/s]

Removing entries from augmented dataset that are different from the base one - based on saved logits:   0%|   …

Tokenizing the provided dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Generating logits for given dataset:   0%|          | 0/421 [00:00<?, ?it/s]

Removing entries from augmented dataset that are different from the base one - based on saved logits:   0%|   …

In [15]:
train_dataset.reset_format()

Ověření, že augmentované věty mají lehce posunutý význam (logity nejsou totožné).

In [16]:
print("First iter check")
print(aug_clean_datasets[1997])
filtered_base = train_dataset.filter(lambda x: x["idx"] == aug_clean_datasets[1997]["idx"])
print(filtered_base[0])

print("Second iter check")
print(aug_clean_datasets[66699])
filtered_base = train_dataset.filter(lambda x: x["idx"] == aug_clean_datasets[66699]["idx"])
print(filtered_base[0])


First iter check
{'idx': 20217, 'sentence': 'will be literally worth your time .', 'labels': 1, 'input_ids': [101, 1209, 1129, 6290, 3869, 1240, 1159, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'logits': [-4.896175384521484, 4.807943344116211]}


Filter:   0%|          | 0/53879 [00:00<?, ? examples/s]

{'idx': 20217, 'sentence': 'will be well worth your time . ', 'labels': 1, 'input_ids': [101, 1209, 1129, 1218, 3869, 1240, 1159, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'logits': [-4.851616859436035, 4.748195171356201]}
Second iter check
{'idx': 30561, 'sentence': 'mr . man and [MASK] . [MASK] [MASK] strong and elysian performances , but', 'labels': 1, 'input_ids': [101, 182, 1197, 119, 1299, 1105, 103, 119, 103, 103, 2012, 1105, 8468, 6834, 1811, 3853, 117, 1133, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'logits': [-3.758352041244507, 3.7316389083862305]}


Filter:   0%|          | 0/53879 [00:00<?, ? examples/s]

{'idx': 30561, 'sentence': 'mr. wollter and ms. seldhal give strong and convincing performances , but ', 'labels': 1, 'input_ids': [101, 182, 1197, 119, 192, 12666, 2083, 1105, 182, 1116, 119, 14516, 5253, 7654, 1660, 2012, 1105, 13870, 3853, 117, 1133, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'logits': [-4.357180595397949, 4.360624313354492]}


In [17]:
print(train_dataset.features)

{'idx': Value(dtype='int32', id=None), 'sentence': Value(dtype='string', id=None), 'labels': ClassLabel(names=['negative', 'positive'], id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'logits': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None)}


Převedení spojeného a zpracovaného augmentovaného datasetu do dataset objektu.

In [18]:
ds_schema = Features({
    "idx": Value("int32"),
    "sentence": Value("string"),
    "labels": ClassLabel(names=["negative", "positive"]),
    "input_ids": Sequence(feature=Value(dtype="int32")),
    "logits": Sequence(feature=Value(dtype="float32")),
})

aug_dataset = Dataset.from_list(aug_clean_datasets)
aug_dataset = aug_dataset.cast(ds_schema)


Casting the dataset:   0%|          | 0/239757 [00:00<?, ? examples/s]

In [ ]:
aug_dataset.set_format(type="torch", columns=["logits", "labels"], device="cpu")
train_dataset.set_format(type="torch", columns=["logits", "labels"], device="cpu")

Výpočet správnosti nad zpracovanými datasety.

In [20]:
print(base.check_acc(train_dataset, "Accuracy for base dataset: "))
print(base.check_acc(aug_dataset, "Accuracy for augmented dataset: "))

Calculating accuracy based on the saved logits:   0%|          | 0/53879 [00:00<?, ?it/s]

Accuracy for base dataset:  0.9888268156424581


Calculating accuracy based on the saved logits:   0%|          | 0/239757 [00:00<?, ?it/s]

Accuracy for augmented dataset:  0.9903151941340608


Spojení původního a augmentovaného datasetu.

In [21]:
train_all_data = concatenate_datasets([train_dataset, aug_dataset])
train_all_data.set_format(type="torch", columns=["logits", "labels"], device="cpu")

Získání správnosti nad touto kombinací.

In [22]:
print(base.check_acc(train_all_data, "Accuracy for combined dataset: "))

Calculating accuracy based on the saved logits:   0%|          | 0/293636 [00:00<?, ?it/s]

Accuracy for combined dataset:  0.9900420929313845


In [23]:
print(train_all_data.column_names)

['idx', 'sentence', 'labels', 'input_ids', 'logits']


Uložení zpracovaných datasetů na disk.

In [24]:
train_all_data.reset_format()
train_all_data = train_all_data.remove_columns(["idx", "input_ids"])

In [25]:
train_all_data.save_to_disk(f"~/data/{DATASET}/train-logits-augmented")

Saving the dataset (0/1 shards):   0%|          | 0/293636 [00:00<?, ? examples/s]

In [26]:
train_dataset.reset_format()
train_dataset = train_dataset.remove_columns(["idx", "input_ids"])
train_dataset.save_to_disk(f"~/data/{DATASET}/train-logits")

Saving the dataset (0/1 shards):   0%|          | 0/53879 [00:00<?, ? examples/s]

Načtení zbylých částí datasetu (validační, umělé testovací a původní testovací). Výpočet logitů je proveden pro každou z těchto částí stejným způsobem jako v příapdě trénovací části.

Nejprve je část tokenizována učitelem a vložena do dataloaderu.

In [27]:
eval_data = load_from_disk(f"~/data/{DATASET}/eval")
eval_dataset = base.prepare_dataset(eval_data, tokenizer)
eval_dataloader = DataLoader(eval_dataset, batch_size=128, shuffle=False)

Tokenizing the provided dataset:   0%|          | 0/872 [00:00<?, ? examples/s]

In [28]:
test_data = load_from_disk(f"~/data/{DATASET}/test")
test_dataset = base.prepare_dataset(test_data, tokenizer)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False)

Tokenizing the provided dataset:   0%|          | 0/13470 [00:00<?, ? examples/s]

In [29]:
test_blank_data = load_from_disk(f"~/data/{DATASET}/test-blank")
test_blank_dataset = base.prepare_dataset(test_blank_data, tokenizer)
test_blank_dataloader = DataLoader(test_blank_dataset, batch_size=128, shuffle=False)

Tokenizing the provided dataset:   0%|          | 0/1821 [00:00<?, ? examples/s]

Následně jsou pro každou část spočteny logity.

In [30]:
eval_logits = base.generate_logits(eval_dataloader, model)
test_logits = base.generate_logits(test_dataloader, model)
test_blank_logits = base.generate_logits(test_blank_dataloader, model)

Generating logits for given dataset:   0%|          | 0/7 [00:00<?, ?it/s]

Generating logits for given dataset:   0%|          | 0/106 [00:00<?, ?it/s]

Generating logits for given dataset:   0%|          | 0/15 [00:00<?, ?it/s]

Logity jsou přidány jako nový sloupec, odstraněny jsou již nepotřebné pozůstatky tokenizace.

In [31]:
eval_dataset.reset_format()
eval_dataset = eval_dataset.add_column("logits", eval_logits)
eval_dataset = eval_dataset.remove_columns(["idx", "token_type_ids", "input_ids", "attention_mask"])

In [32]:
test_dataset.reset_format()
test_dataset = test_dataset.add_column("logits", test_logits)
test_dataset = test_dataset.remove_columns(["idx", "token_type_ids", "input_ids", "attention_mask"])

In [33]:
test_blank_dataset.reset_format()
test_blank_dataset = test_blank_dataset.add_column("logits", test_blank_logits)
test_blank_dataset = test_blank_dataset.remove_columns(["idx", "token_type_ids", "input_ids", "attention_mask"])

Uložení dat na disk.

In [34]:
eval_dataset.save_to_disk(f"~/data/{DATASET}/eval-logits")
test_dataset.save_to_disk(f"~/data/{DATASET}/test-logits")
test_blank_dataset.save_to_disk(f"~/data/{DATASET}/test-blank-logits")

Saving the dataset (0/1 shards):   0%|          | 0/872 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/13470 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1821 [00:00<?, ? examples/s]

Vypočtení správnosti nad validační a umělou testovací částí datasetu.

In [35]:
eval_data = load_from_disk(f"~/data/{DATASET}/eval-logits")
test_data = load_from_disk(f"~/data/{DATASET}/test-logits")

eval_data.set_format(type="torch", columns=["logits", "labels"], device="cpu")
test_data.set_format(type="torch", columns=["logits", "labels"], device="cpu")

print(base.check_acc(eval_data, "Accuracy for base eval dataset: "))
print(base.check_acc(test_data, "Accuracy for base test dataset: "))

Calculating accuracy based on the saved logits:   0%|          | 0/872 [00:00<?, ?it/s]

Accuracy for base eval dataset:  0.9231651376146789


Calculating accuracy based on the saved logits:   0%|          | 0/13470 [00:00<?, ?it/s]

Accuracy for base test dataset:  0.9896807720861173


Výpočet výkonnostních metrik a velikosti u učitelského modelu.

In [38]:
train_dataset = base.prepare_dataset(train_data, tokenizer)

train_data_gpu = copy.deepcopy(train_dataset)
train_data_gpu.set_format(type="torch", columns=["input_ids", "attention_mask"], device="cuda")
gpu_data_loader = DataLoader(train_data_gpu, batch_size=1, shuffle=False)

train_data_cpu = copy.deepcopy(train_dataset)
train_data_cpu.set_format(type="torch", columns=["input_ids", "attention_mask"], device="cpu")
cpu_data_loader = DataLoader(train_data_cpu, batch_size=1, shuffle=False)

In [39]:
base.count_parameters(model)

model size: 413.185MB.
Total Trainable Params: 108311810.


,Modules,Parameters
0,bert.embeddings.word_embeddings.weight,22268928
1,bert.embeddings.position_embeddings.weight,393216
2,bert.embeddings.token_type_embeddings.weight,1536
3,bert.embeddings.LayerNorm.weight,768
4,bert.embeddings.LayerNorm.bias,768
...,...,...
196,bert.encoder.layer.11.output.LayerNorm.bias,768
197,bert.pooler.dense.weight,589824
198,bert.pooler.dense.bias,768
199,classifier.weight,1536


In [45]:
cpu_benchmark = base.BenchMarkRunner(model, cpu_data_loader, "cpu", 1000)
print(cpu_benchmark.run_benchmark())

self.infer_speed_comp()
  50.36 ms
  1 measurement, 1000 runs , 4 threads


In [46]:
gpu_benchmark = base.BenchMarkRunner(model, gpu_data_loader, "cuda", 1000)
print(gpu_benchmark.run_benchmark())

self.infer_speed_comp()
  6.02 ms
  1 measurement, 1000 runs , 4 threads


In [44]:
base.get_scores(test_data)

F1 score: 0.9895478534592006
Accuracy: 0.9896807720861173
Precision: 0.989232666017668
Recall: 0.9898802736347267
